In [1]:
import tensorflow as tf
import utils
import numpy as np
import random as rn
import os

In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('.','data','royalty.npz'))

triples = data['triples']
traces = data['traces']
entities = data['entities'].tolist()
num_entities = len(entities)
relations = data['relations'].tolist()
num_relations = len(relations)
embedding_dim = 3
ent2idx = dict(zip(entities, range(num_entities)))
rel2idx = dict(zip(relations, range(num_relations)))

In [3]:
train2idx = utils.array2idx(triples, ent2idx,rel2idx)

In [4]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import RandomUniform
from tensorflow.python.ops import embedding_ops
import tensorflow.keras.backend as K

entity_embeddings = Embedding(
    input_dim=num_entities,
    output_dim=embedding_dim,
    name='entity_embeddings',
    embeddings_initializer=RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
        )
    )

In [5]:
#np.ones((1,3,3))*np.eye(3)

In [6]:
head_input = tf.keras.Input(shape=(), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(), name='tail_input',dtype=tf.int64)
#all_entities = tf.keras.Input(shape=(None,),name='all_input')

In [7]:
head_e = entity_embeddings(head_input)
#rel_e = relation_embeddings(rel_input)
tail_e = entity_embeddings(tail_input)
#all_e = entity_embeddings(all_entities)

In [8]:
class RGCN(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,*args,**kwargs):
        super(RGCN,self).__init__(*args,**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
    
    def build(self,input_shape):

        in_shape = input_shape[-1][-1]

        self.W_r = self.add_weight(
            shape=(self.num_relations,self.output_dim,in_shape),
            trainable=True,
            initializer="random_normal",
            name='W_r'
        )
        
        self.W0 = self.add_weight(
            shape=(self.output_dim,in_shape),
            trainable=True,
            initializer='random_normal',
            name='W0'
        )
        
    def call(self,inputs):
        
        head_input,tail_input = inputs
        
        tail_update = K.dot(self.W_r,K.transpose(tail_input))
        
        head_update = K.dot(self.W0,K.transpose(head_input))
        
        update = K.sum(tail_update + head_update, axis=0)

        return K.transpose(update)

In [9]:
class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,*args,**kwargs):
        super(DistMult,self).__init__(*args,**kwargs)
        self.num_relations = num_relations
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer='random_normal',
            name='rel_embedding'
        )
        
#     def call(self,inputs):
#         #must be of dimensions (batch size,)
#         embeddings,head_idx,rel_idx, tail_idx = inputs
        
#         updated_head_e = embedding_ops.embedding_lookup_v2(embeddings,head_idx)
#         rel_e = embedding_ops.embedding_lookup_v2(self.kernel,rel_idx)
#         updated_tail_e = embedding_ops.embedding_lookup_v2(embeddings,tail_idx)

#         return K.sum(updated_head_e*rel_e*updated_tail_e, axis=-1)
    def call(self,inputs):
        
        head_e,rel_idx,tail_e = inputs
        
        rel_e = embedding_ops.embedding_lookup_v2(self.kernel,rel_idx)
        
        return tf.reduce_sum(head_e*rel_e*tail_e, axis=-1)

In [10]:
new_head = RGCN(num_relations=num_relations,output_dim=5)([head_e,tail_e])
new_head = tf.keras.layers.Activation('sigmoid')(new_head)

new_tail = RGCN(num_relations=num_relations,output_dim=5)([tail_e,head_e])
new_tail = tf.keras.layers.Activation('sigmoid')(new_tail)

output = DistMult(num_relations=num_relations)([new_head,rel_input,new_tail])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [11]:
model = tf.keras.Model([head_input,rel_input,tail_input],[output])

In [12]:
model.compile(loss='binary_crossentropy', optimizer='sgd')

In [13]:
model.fit(x=[train2idx[:,0],train2idx[:,1],train2idx[:,2]],y=np.ones(train2idx.shape[0]),epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
288/288 [==============================] - 0s 741us/step - loss: 0.4069


In [ ]:
#np.dot(np.ones((4,5,3)),np.zeros((3,100))).sum(axis=0).shape

In [ ]:
#np.dot(np.ones((5,3)),np.zeros((3,100))).shape

In [ ]:
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
import dgl
from dgl.nn import RelGraphConv
g = dgl.graph(([0,1,2,3,2,5,0], [1,2,3,4,0,3,1]))

In [ ]:
conv = RelGraphConv(10, 5, 3, regularizer='basis')

In [ ]:
import torch as th
feat = th.ones(6, 10)
etype = th.tensor(np.array([0,1,2,0,1,2,1]).astype(np.int64))

In [ ]:
conv(g, feat, etype)